In [1]:
import logging
import orjson
import numpy as np
import math
from simpletransformers.question_answering import QuestionAnsweringModel
from os import cpu_count

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [2]:
with open("../resources/data_final.json", "rb") as f:
    dataset = orjson.loads(f.read())

In [3]:
original = dataset.copy()

In [4]:
np.random.seed(0)
id_train = np.random.choice(len(dataset), math.ceil(len(dataset)/2))
id_test = np.asarray(list(set(range(len(dataset))).difference(set(id_train))))

In [5]:
# Create dummy data to use for training.
train_data = [dataset[id] for id in id_train].copy()
test_data = [dataset[id] for id in id_test].copy()

In [6]:
args = {
    "output_dir": "../resources/bert/",
    "cache_dir": "../resources/cache/",
    "best_model_dir": "../resources/best_model/",

    "fp16": True,
    "max_seq_length": 64,
    "train_batch_size": 4,
    "eval_batch_size": 4,
    "gradient_accumulation_steps": 6,
    "num_train_epochs": 200,
    "weight_decay": 0,
    "learning_rate": 3e-5,
    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,
    "do_lower_case": True,
    "logging_steps": 50,
    "evaluate_during_training": True,
    "evaluate_during_training_steps": 2000,
    "evaluate_during_training_verbose": True,
    "use_cached_eval_features": False,
    "save_eval_checkpoints": True,
    "save_steps": 100,
    "no_cache": False,
    "save_model_every_epoch": True,
    "tensorboard_dir": None,

    "overwrite_output_dir": True,
    "reprocess_input_data": True,

    "process_count": cpu_count() - 2 if cpu_count() > 2 else 1,
    "n_gpu": 1,
    "silent": False,
    "use_multiprocessing": True,

    "wandb_project": None,
    "wandb_kwargs": {},

    "use_early_stopping": True,
    "early_stopping_patience": 3,
    "early_stopping_delta": 0,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,

    "manual_seed": 0,
    "encoding": None,
}

In [7]:
# Create the QuestionAnsweringModel
model = QuestionAnsweringModel('bert',
                               '../resources/bert/checkpoint-316-epoch-158',
                               args=args)
model.lazy_loading = True

model.train_model(train_data,eval_data=test_data)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/48 [00:00<?, ?it/s]/home/giuseppe/PycharmProjects/ple_chat_bot/venv/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 48/48 [00:00<00:00, 85489.00it/s]
INFO:simpletransformers.question_answering.question_answering_model:   Continuing training from checkpoint, will skip to saved global_step
INFO:simpletransformers.question_answering.question_answering_model:   Continuing training from epoch 158
INFO:simpletransformers.question_answering.question_answering_model:   Continuing training from global step 316
INFO:simpletransformers.question_answering.question_answering_model:   Will skip the first 0 steps in the curren

In [8]:
# Making predictions using the model.
to_predict = np.asarray([dataset[0]])
predict = model.predict(to_predict, n_best_size=1)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
convert squad examples to features:   0%|          | 0/1 [00:00<?, ?it/s]/home/giuseppe/PycharmProjects/ple_chat_bot/venv/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1324: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 6204.59it/s]
